In [1]:
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler

In [ ]:
# extract features
feature_density = False
feature_speed = True
feature_acceleration = False
feature_heading = False

In [2]:
# load the data
taxi_trajectories =pd.read_csv("../../datasets/T-Drive/taxi_trajectories.csv")
# load intersections bound box
intersection_boundBox = pd.read_csv("..")

In [93]:
for inter_num in range(len(intersection_boundBox)):
    # extract intersection bound box and points
    min_longitude = intersection_boundBox['min_lon'][inter_num]
    max_longitude = intersection_boundBox['max_lon'][inter_num]
    min_latitude = intersection_boundBox['min_lat'][inter_num]
    max_latitude = intersection_boundBox['max_lat'][inter_num]
    intersection_points = taxi_trajectories.loc[(taxi_trajectories['longitude']<max_longitude)&
                                                (taxi_trajectories['longitude']>min_longitude)&
                                                (taxi_trajectories['latitude']<max_latitude)&
                                                (taxi_trajectories['latitude']>min_latitude)]
    # the grid dimensions[#cells = cells_columns_number * cells_row_number]
    cells_columns_number = 14
    cells_row_number = 14
    # get the steps values
    longitude_step = np.abs(max_longitude - min_longitude) / (cells_row_number*2)
    latitude_step = np.abs(max_latitude - min_latitude) / (cells_columns_number*2)
    cells_center_matrix = np.zeros((cells_row_number, cells_columns_number, 2))
    # computer the center points
    for i in range(cells_row_number):
        for j in range(cells_columns_number):
            cells_center_matrix[i,j,0] = max_longitude - (1+2*i) * longitude_step      # lonitude -> X0
            cells_center_matrix[i,j,1] = min_latitude + (1+2*j) * latitude_step        # latitude -> Y0
    # Grid       
    density_Grid = np.zeros((cells_row_number, cells_columns_number))
    speed_Grid = np.zeros((cells_row_number, cells_columns_number))
    acceleration_Grid = np.zeros((cells_row_number, cells_columns_number))
    head_Grid = np.zeros((cells_row_number, cells_columns_number))
    # gaussain kernel
    def gaussain_kernel(x0, y0, xi, yi, sigma):
        constant_part= 1 / (np.sqrt(2 * np.pi * sigma))
        expon_part=np.exp(-1 * (np.power((xi-x0),2) + np.power((yi-y0),2)) / (2 * np.power(sigma,2)))
        w = constant_part * expon_part
        return w
    # compute Grid features 
    sigma = 0.0001
    for i in range(cells_row_number):
        for j in range(cells_columns_number):
            x_i = np.array(intersection_points['longitude'])
            y_i = np.array(intersection_points['latitude'])
            x_0 = np.array([cells_center_matrix[i,j,0]])
            y_0 = np.array([cells_center_matrix[i,j,1]])
            distance = np.sqrt(np.power((x_0 - x_i),2) + np.power((y_0 - y_i),2))
            x_i = x_i[distance<=(3*sigma)]
            y_i = y_i[distance<=(3*sigma)] 
            weights = gaussain_kernel(x_0 , y_0, x_i, y_i, sigma)
            if(feature_density):
                density_Grid[i,j] = np.sum(weights)
            if(feature_speed):
                speed = np.array(intersection_points['speed'])
                speed = speed[distance<=(3*sigma)]    
                speed_Grid[i,j] = np.dot(weights, speed)
            if(feature_acceleration):
                acceleration = np.array(intersection_points['acceleration'])
                acceleration = acceleration[distance<=(3*sigma)]        
                acceleration_Grid[i,j] = np.dot(weights, acceleration)
            if(feature_heading):     
                head = np.array(intersection_points['head'])
                head = head[distance<=(3*sigma)]        
                head_Grid[i,j] = np.dot(weights, head)

    # Define image dimension
    image_dims = 224
    image = np.zeros((image_dims,image_dims))
    x_steps = image_dims / cells_columns_number
    y_steps = image_dims / cells_row_number
    feature_image = np.zeros((image_dims,image_dims,3))
    channel = 0
    image_name = ''
    # build image channels
    if(feature_density):
        for i in range(image_dims):
            for j in range(image_dims):
                image[i,j] = density_Grid[int(i/x_steps),int(j/y_steps)]
        image = image.T                 # to get the image without reverse
        image = np.flip(image, -1)      # or fliping
        image = np.flip(image, 0)
        scaler = MinMaxScaler()
        image_s = scaler.fit_transform(image)
        # this part because of the problem of 1.0000000000000002 and must be 0-1 for RGB
        for i in range(len(image_s)):
            for j in range(len(image_s[i])):
                if(image_s[i,j]>1):
                    image_s[i,j] = 1.0    
        feature_image[:,:,channel] = image_s  
        channel += 1
        image_name += 'D'
    if(feature_speed):
        for i in range(image_dims):
            for j in range(image_dims):
                image[i,j] = speed_Grid[int(i/x_steps),int(j/y_steps)]
        image = image.T                 # to get the image without reverse
        image = np.flip(image, -1)      # or fliping
        image = np.flip(image, 0)
        scaler = MinMaxScaler()
        image_s = scaler.fit_transform(image)
        # this part because of the problem of 1.0000000000000002 and must be 0-1 for RGB
        for i in range(len(image_s)):
            for j in range(len(image_s[i])):
                if(image_s[i,j]>1):
                    image_s[i,j] = 1.0    
        feature_image[:,:,channel] = image_s  
        channel += 1
        image_name += 'S'                            
    if(feature_acceleration):
        for i in range(image_dims):
            for j in range(image_dims):
                image[i,j] = acceleration_Grid[int(i/x_steps),int(j/y_steps)]
        image = image.T                 # to get the image without reverse
        image = np.flip(image, -1)      # or fliping
        image = np.flip(image, 0)
        scaler = MinMaxScaler()
        image_s = scaler.fit_transform(image)
        # this part because of the problem of 1.0000000000000002 and must be 0-1 for RGB
        for i in range(len(image_s)):
            for j in range(len(image_s[i])):
                if(image_s[i,j]>1):
                    image_s[i,j] = 1.0    
        feature_image[:,:,channel] = image_s  
        channel += 1 
        image_name += 'A'
    if(feature_heading):
        for i in range(image_dims):
            for j in range(image_dims):
                image[i,j] = head_Grid[int(i/x_steps),int(j/y_steps)]
        image = image.T                 # to get the image without reverse
        image = np.flip(image, -1)      # or fliping
        image = np.flip(image, 0)
        scaler = MinMaxScaler()
        image_s = scaler.fit_transform(image)
        # this part because of the problem of 1.0000000000000002 and must be 0-1 for RGB
        for i in range(len(image_s)):
            for j in range(len(image_s[i])):
                if(image_s[i,j]>1):
                    image_s[i,j] = 1.0    
        feature_image[:,:,channel] = image_s  
        channel += 1
        image_name += 'H'
    # extract image    
    image_name = str(intersection_boundBox['label'][inter_num]) + str(inter_num) + '_' + image_name + '.jpg'    
    plt.imsave(image_name,feature_image,format='jpg')                         